# Obligations in District 3

Using data from the Division of Local Assistance [Obligation list](https://dot.ca.gov/programs/local-assistance/reports/e-76-obligated)

In [2]:
import numpy as np
import pandas as pd
from siuba import *

import altair as alt
import altair_saver
from plotnine import *

from IPython.display import Markdown

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

from calitp import to_snakecase
import intake

import clean_data
import _dla_utils

In [37]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
#df = _clean_data.make_clean_data()
df= pd.read_parquet("dla_df.parquet")

In [4]:
#parameters cell
subset = 3

In [5]:
df.sample()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
582,Obligated,ER,30E0(021),Calaveras County,2018-08-23,2018-08-23,2018-09-12,2018-09-13,2018-09-14,18345.0,...,nan,Sheep Ranch Road Mp 0.0 At Mountain Ranch Rd,Remove And Replace Damaged Ac Pavement. Temp ...,1,2018-08-21,2018-08-21,NON-MPO,30.0,2018.0,Calaveras County


In [6]:
df = (df>>filter(_.dist==subset))

In [7]:
df_years = _dla_utils.count_all_years(df)

In [8]:
df_years

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_mpo,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
8,2021,3,1.035237e+08,1.089277e+08,2.378141e+08,417434.091169,439224.708387,9.589276e+05,5.0,34.0,45.0,191.0,198.0,164.0
7,2020,3,-6.858257e+06,1.551299e+08,2.389182e+08,-22486.088393,508622.595902,7.833385e+05,5.0,38.0,43.0,227.0,234.0,189.0
6,2019,3,-1.033257e+07,1.800790e+08,2.115672e+08,-25831.428050,450197.418575,5.289181e+05,4.0,51.0,46.0,316.0,321.0,254.0
0,2018,3,2.527674e+07,1.352713e+08,1.904728e+08,73693.127668,394376.846851,5.553144e+05,4.0,38.0,47.0,277.0,282.0,219.0
1,2017,3,4.843487e+07,1.713219e+08,2.014909e+08,156747.156019,554439.921909,6.520742e+05,4.0,42.0,44.0,266.0,270.0,220.0
2,2016,3,7.230150e+05,1.267155e+08,1.623934e+08,2362.794118,414102.874412,5.306974e+05,4.0,44.0,47.0,257.0,269.0,218.0
3,2015,3,-3.169509e+06,9.522479e+07,1.143066e+08,-11652.606618,350091.136213,4.202447e+05,4.0,36.0,46.0,223.0,232.0,184.0
4,2014,3,-8.798279e+06,1.343414e+08,1.824704e+08,-27580.811912,421132.762978,5.720075e+05,6.0,34.0,43.0,254.0,264.0,200.0
5,2013,3,2.100000e+05,6.672364e+06,2.177725e+07,17500.000000,556030.362500,1.814771e+06,2.0,11.0,10.0,12.0,12.0,11.0


In [9]:
df_top = _dla_utils.find_top(df)

In [10]:
df_top

,value,count,variable
0,CML,396,prefix
1,BRLO,384,prefix
2,HSIPL,378,prefix
3,STPL,276,prefix
4,ER,158,prefix
...,...,...,...
16,Placer County Transportation Planning Agency,43,primary_agency_name
17,Folsom,41,primary_agency_name
18,Glenn County,38,primary_agency_name
19,Lincoln,37,primary_agency_name


## Which Agencies have Obligations

### How many Unique Agencies

In [11]:
_dla_utils.calculate_data_all(df, 'primary_agency_name', aggfunc="nunique")

,dist,primary_agency_name
0,3,62


In [12]:
df>>count(_.primary_agency_name)>>arrange(-_.n)

,primary_agency_name,n
15,El Dorado County,293
42,Sacramento County,221
40,Sacramento,182
34,Placer County,173
61,Yuba County,141
...,...,...
5,California Tahoe Conservancy,2
26,Marysville,2
14,Dixon,1
28,Natomas Unified School District,1


### Agencies with the most Obligations

In [13]:
df_top>>filter(_.variable == 'primary_agency_name')

,value,count,variable
0,El Dorado County,293,primary_agency_name
1,Sacramento County,221,primary_agency_name
2,Sacramento,182,primary_agency_name
3,Placer County,173,primary_agency_name
4,Yuba County,141,primary_agency_name
5,Nevada County,139,primary_agency_name
6,Butte County,133,primary_agency_name
7,Yolo County,89,primary_agency_name
8,Elk Grove,64,primary_agency_name
9,Sierra County,61,primary_agency_name


### Agencies with the least Obligations

In [14]:
(df>>count(_.primary_agency_name)>>arrange(-_.n)).tail(10)

,primary_agency_name,n
27,Modoc County,4
51,"U.S. Forest Service, Pacific Southwest Region",4
52,University of California - Davis,4
7,Capital Southeast Connector,3
31,Orland,3
5,California Tahoe Conservancy,2
26,Marysville,2
14,Dixon,1
28,Natomas Unified School District,1
48,Tahoe Regional Planning Agency,1


## Transit Funding

In [81]:
transit = df[df['type_of_work'].str.contains(
              'Transit|tranist|bus|Bus|Rideshare',
              case=False, na=False)]

In [93]:
print(f"Out of {len(df)} obligations, {len(transit)} are transit related")

Out of 2517 obligations, 21 are transit related


In [83]:
transit>>count(_.primary_agency_name)>>arrange(-_.n)

,primary_agency_name,n
4,Tahoe Transportation District,10
0,Caltrans,3
1,Chico,2
2,Citrus Heights,2
3,Sacramento Area Council of Governments,2
5,West Sacramento,2


In [84]:
#transit>>group_by(_.prefix)>>count(_.type_of_work)>>arrange(-_.n)

In [85]:
transit>>count(_.type_of_work)>>arrange(-_.n)

,type_of_work,n
5,"Realign Roadway, Reduce Lanes And Transit-bike...",5
2,Bus/carpool Lane/comm. Enhancm,2
3,Operational And Safety Improvements To The Heg...,2
4,"Pa&ed Only For New Bridge - Includes Auto, Bik...",2
6,Spare The Air / Free Bus Rides (non-infrastruc...,2
8,Transit Operating Assistance - Develop A Proje...,2
9,Universal Transit Fare Card Pr,2
0,Analyze Transit Service And Asset Management F...,1
1,Analyze Transit Service And Asset Management F...,1
7,Transit Operating Assistance - Develop A Proje...,1


In [32]:
transit>>count(_.prefix)>>arrange(-_.n)

,prefix,n
4,STPLN,6
1,CML,4
5,STPLNI,3
0,CMHIPL,2
2,CMLN,2
3,STPL,2
6,TGR2DG,1
7,TGR2DGL,1


In [33]:
transit_agencies = df[df['primary_agency_name'].str.contains(
              'Transit|tranist|Rail',
              case=False, na=False)]

In [34]:
transit_agencies>>count(_.primary_agency_name)

,primary_agency_name,n
0,Sacramento Regional Transit District,30


In [35]:
transit_agencies>>count(_.type_of_work)>>arrange(-_.n)
#using this to determine how many FTA transfers there are  

,type_of_work,n
0,FTA Transfer,30


In [38]:
_dla_utils.calculate_data_all(transit_agencies, 'total_requested', aggregate_by='primary_agency_name', aggfunc="mean")

,primary_agency_name,total_requested
0,Sacramento Regional Transit District,"2,585,441.17"


### Charting

In [39]:
_dla_utils.basic_bar_chart((df_top>>filter(_.variable=='primary_agency_name')), 'value', 'count', 'value', subset, 'Agencies With The Most Obligations')

alt.Chart(...)

In [40]:
df_top>>filter(_.variable=='prefix')

,value,count,variable
0,CML,396,prefix
1,BRLO,384,prefix
2,HSIPL,378,prefix
3,STPL,276,prefix
4,ER,158,prefix
5,BRLS,151,prefix
6,RPSTPL,65,prefix
7,HRRRL,60,prefix
8,BPMP,50,prefix
9,ACSTP,43,prefix


In [90]:
_dla_utils.basic_bar_chart((df_top>>filter(_.variable=='prefix')), 'value', 'count', 'value', subset, 'Most used Prefix Codes')

alt.Chart(...)

In [102]:
(df>>group_by(_.primary_agency_name)>>count(_.prefix)>>arrange(-_.n))

,primary_agency_name,prefix,n
71,El Dorado County,BRLO,67
141,Nevada County,BRLO,63
162,Placer County,BRLO,58
232,Sacramento,HSIPL,54
260,Sacramento County,HSIPL,53
...,...,...,...
316,Winters,HPLUL,1
319,Woodland,ATPSB1L,1
323,Woodland,RPSTPL,1
328,Yolo County,BRLOZE,1


## Number of Unique Prefix Codes

In [109]:
(_dla_utils.get_nunique(df, 'prefix', 'primary_agency_name'))

,primary_agency_name,n
15,El Dorado County,21
40,Sacramento,18
42,Sacramento County,18
34,Placer County,14
2,Butte County,13
...,...,...
28,Natomas Unified School District,1
31,Orland,1
48,Tahoe Regional Planning Agency,1
52,University of California - Davis,1


In [115]:
(_dla_utils.get_nunique(df, 'primary_agency_name', 'prefix'))

,prefix,n
21,CML,35
85,STPL,27
57,HSIPL,24
72,RPSTPL,17
14,BRLO,16
...,...,...
92,STPLR,1
93,STPLZ,1
95,TGR2DG,1
96,TGR2DGL,1


In [43]:
_dla_utils.basic_bar_chart(((_dla_utils.get_nunique(df, 'prefix', 'primary_agency_name')).head(30)),
                            'primary_agency_name', 'n', 'primary_agency_name', subset, 'Agencies With The Most Unique Prefix Codes')

alt.Chart(...)

In [88]:
df_top>>filter(_.variable=="prefix")

,value,count,variable
0,CML,396,prefix
1,BRLO,384,prefix
2,HSIPL,378,prefix
3,STPL,276,prefix
4,ER,158,prefix
5,BRLS,151,prefix
6,RPSTPL,65,prefix
7,HRRRL,60,prefix
8,BPMP,50,prefix
9,ACSTP,43,prefix


## Most Common Types of Work by Agnecy

In [44]:
_dla_utils.get_nunique(df, 'primary_agency_name', 'type_of_work')

,type_of_work,n
89,Bridge Replacement,17
90,Bridge Replacement (tc),15
307,FTA Transfer,10
82,Bridge Rehabilitation,6
83,Bridge Rehabilitation (tc),6
...,...,...
818,Widen Travel Lanes; Construct Drainage (tc),1
819,Widen Travel Lanes; Construct Drainage Imprvmnts,1
820,"Widen/aux. Lanes, On/off Ramp",1
821,"Widening, Trainl And Intersection Improvements",1


In [45]:
_dla_utils.calculate_data_head(df, "primary_agency_name",'type_of_work', aggfunc="count")

,type_of_work,primary_agency_name
90,Bridge Replacement (tc),250
89,Bridge Replacement,126
307,FTA Transfer,57
82,Bridge Rehabilitation,22
83,Bridge Rehabilitation (tc),22
441,New Engineered Embankment System Will Need To ...,18
49,Bike Path (tc),11
275,Emergency Opening,10
513,Preventative Maintenance,9
488,Pedestrian And Bike Path,9


In [46]:
df>>group_by(_.primary_agency_name)>>count(_.type_of_work)>>arrange(-_.n)>>filter(_.n>=8)

,primary_agency_name,type_of_work,n
148,El Dorado County,Bridge Replacement (tc),45
718,Sacramento Regional Transit District,FTA Transfer,30
863,Yuba County,Bridge Replacement (tc),29
315,Nevada County,Bridge Replacement (tc),28
821,Yolo County,Bridge Replacement (tc),27
381,Placer County,Bridge Replacement (tc),25
147,El Dorado County,Bridge Replacement,22
633,Sacramento County,Bridge Replacement (tc),21
749,Sutter County,Bridge Replacement (tc),20
380,Placer County,Bridge Replacement,19


## Most Common Project Locations

In [47]:
df_top>>filter(_.variable=='project_location')

,value,count,variable
0,Sacog Region,29,project_location
1,Various Locations (see Comments),10,project_location
2,5th Street At 2nd Street And 5th Street Over F...,9,project_location
3,4 Corridors In Western Nevada County,8,project_location
4,Along Newtown Road Between Hwy 49 And Beckvill...,8,project_location
5,El Dorado Trail From Los Trampas Dr To Halcon Rd,8,project_location
6,Nevada City Highway Starting At Banner Lava Ca...,8,project_location
7,Pctpa Region,8,project_location
8,Bradshaw & Sheldon At Laguna Crk Br #24c-0308,7,project_location
9,Ed Hills Csd From The West End Of Ny Trail Eas...,7,project_location


## Number of Obligations by Year

In [48]:
df_top>>filter(_.variable=='prepared_y')

,value,count,variable
0,"2,019.00",400,prepared_y
1,"2,018.00",343,prepared_y
2,"2,014.00",319,prepared_y
3,"2,017.00",309,prepared_y
4,"2,016.00",306,prepared_y
5,"2,020.00",305,prepared_y
6,"2,015.00",272,prepared_y
7,"2,021.00",248,prepared_y
8,"2,013.00",12,prepared_y


In [49]:
df>>group_by(_.primary_agency_name)>>count(_.prepared_date)>>arrange(-_.n)

,primary_agency_name,prepared_date,n
1617,Sacramento County,2018-10-18,9
1739,Sacramento Regional Transit District,2020-04-16,8
888,Nevada County,2020-10-08,7
863,Nevada County,2019-03-20,6
1357,Sacramento,2016-04-29,6
...,...,...,...
2211,Yuba County,2021-04-30,1
2212,Yuba County,2021-06-30,1
2213,Yuba County,2021-08-05,1
2214,Yuba County,2021-08-19,1


### Chart

In [50]:
_dla_utils.basic_line_chart((df_top>>filter(_.variable=='prepared_y')), 'value', 'count', subset, 'Obligations by Year')

alt.Chart(...)

## Agencies with the most unique project numbers

In [51]:
df>>group_by(_.primary_agency_name, _.prefix)>>count(_.project_no)>>arrange(-_.n)>>filter(_.n>5)

,primary_agency_name,prefix,project_no,n
370,Nevada County,BRLO,5917(081),9
941,Yuba City,BRLS,5163(027),9
172,El Dorado County,BRLO,5925(095),8
195,El Dorado County,CML,5925(125),8
197,El Dorado County,CML,5925(130),8
250,Elk Grove,BRLS,5479(012),8
369,Nevada County,BRLO,5917(080),8
371,Nevada County,BRLO,5917(082),8
378,Nevada County,CML,5917(070),8
379,Nevada County,CML,5917(090),8


## Funding Amounts

In [52]:
_dla_utils.calculate_data_all(df, 'total_requested', 'primary_agency_name', aggfunc="mean")

,primary_agency_name,total_requested
44,Sacramento Regional Transit District,"2,585,441.17"
60,Yuba City,"2,415,526.06"
52,University of California - Davis,"1,916,020.75"
57,Woodland,"1,854,886.80"
26,Marysville,"1,609,720.50"
59,Yolo County Transportation District,"1,541,053.60"
17,Folsom,"1,239,564.15"
43,Sacramento Metropolitan Air Quality Management...,"1,187,538.65"
42,Sacramento County,"1,068,964.51"
53,West Sacramento,"1,007,069.30"


In [119]:
_dla_utils.calculate_data_all(df, 'total_requested', 'type_of_work', aggfunc="mean")

,type_of_work,total_requested
149,Construct 4 Lane Road With 8 Foot Shoulders. ...,"31,201,500.00"
534,Realign And Widen From 2-4 Lanes And Signalize...,"22,278,000.00"
329,Hov Lanes And I/c Reconstructi,"15,065,434.00"
639,Road Widening And Ped/bike Facilities,"10,296,788.06"
99,Bridge Replacement. Toll Credits For Stbgp Fun...,"9,962,583.50"
...,...,...
760,Traffic Safety Educational Pro,"2,259.00"
290,Emergency Opening And Restoration. Damage To ...,"2,003.46"
129,Cmaq Program Admin,30.53
728,Spare The Air Voluntary Driving Curtailment Pr...,0.35


In [53]:
pd.options.display.float_format = '{:,.2f}'.format

In [94]:
df.describe()

,fed_requested,ac_requested,total_requested,dist,dist_processing_days,hq_processing_days,fhwa_processing_days,seq,prepared_y
count,"2,517.00","2,517.00","2,517.00","2,517.00","2,306.00","2,495.00","2,495.00","2,517.00","2,514.00"
mean,"442,762.63","59,201.30","620,564.43",3.00,16.02,12.99,4.72,3.23,"2,017.48"
std,"1,431,678.51","2,314,056.64","3,107,301.55",0.00,74.31,17.89,4.27,2.05,2.23
min,"-7,338,664.05","-18,714,357.00","-52,762,263.75",3.00,-90.00,0.00,0.00,1.00,"2,013.00"
25%,0.00,0.00,0.00,3.00,0.00,1.00,2.00,2.00,"2,016.00"
50%,"10,442.00",0.00,"60,000.00",3.00,3.00,8.00,4.00,3.00,"2,018.00"
75%,"312,000.00",0.00,"432,001.00",3.00,12.00,16.50,6.00,4.00,"2,019.00"
max,"24,669,621.91","70,500,000.00","70,500,000.00",3.00,"3,177.00",177.00,45.00,13.00,"2,021.00"


In [95]:
transit.describe()

,fed_requested,ac_requested,total_requested,dist,dist_processing_days,hq_processing_days,fhwa_processing_days,seq,prepared_y
count,21.00,21.00,21.00,21.00,20.00,21.00,21.00,21.00,21.00
mean,"669,953.33","-18,636.81","730,728.08",3.00,11.25,16.95,6.29,2.67,"2,018.00"
std,"1,106,214.66","88,147.85","1,426,353.04",0.00,19.96,27.53,6.30,1.83,2.30
min,"-36,277.43","-391,373.00","-2,206,077.53",3.00,0.00,0.00,0.00,1.00,"2,014.00"
25%,0.00,0.00,0.00,3.00,0.00,0.00,3.00,1.00,"2,016.00"
50%,"183,300.00",0.00,"66,645.00",3.00,1.50,12.00,5.00,2.00,"2,018.00"
75%,"1,041,000.00",0.00,"1,110,000.00",3.00,10.00,19.00,8.00,4.00,"2,020.00"
max,"4,000,000.00","69,000.00","4,000,000.00",3.00,77.00,125.00,29.00,7.00,"2,021.00"


### Top 50 Agencies with highest funds

In [56]:
((_dla_utils.calculate_data_all(df, 'total_requested', 'primary_agency_name', aggfunc="mean"))>>arrange(-_.total_requested)).head(50)

,primary_agency_name,total_requested
44,Sacramento Regional Transit District,"2,585,441.17"
60,Yuba City,"2,415,526.06"
52,University of California - Davis,"1,916,020.75"
57,Woodland,"1,854,886.80"
26,Marysville,"1,609,720.50"
59,Yolo County Transportation District,"1,541,053.60"
17,Folsom,"1,239,564.15"
43,Sacramento Metropolitan Air Quality Management...,"1,187,538.65"
42,Sacramento County,"1,068,964.51"
53,West Sacramento,"1,007,069.30"


In [57]:
_dla_utils.basic_bar_chart((((_dla_utils.calculate_data_all(df, 'total_requested', 'primary_agency_name', aggfunc="mean"))
                          >>arrange(-_.total_requested)).head(30)
                        ), 'primary_agency_name','total_requested', 'primary_agency_name', subset,
                           'Average Total Requested Funds by Agency'
                       )

alt.Chart(...)

In [58]:
_dla_utils.basic_bar_chart((((_dla_utils.calculate_data_all(transit, 'total_requested', 'primary_agency_name', aggfunc="mean"))
                          >>arrange(-_.total_requested)).head(30)
                        ), 'primary_agency_name','total_requested', 'primary_agency_name', subset,
                           'Average Total Requested Funds by Agency: Transit Related Funding'
                       )

alt.Chart(...)

In [59]:
_dla_utils.basic_bar_chart((_dla_utils.calculate_data_all(transit_agencies, 'total_requested', aggregate_by='primary_agency_name', aggfunc="mean")),
                           'primary_agency_name', 'total_requested', 'primary_agency_name', subset,
                           'Transit Agencies Average Funding')



alt.Chart(...)

In [60]:
df>>filter(_.primary_agency_name=='San Joaquin Regional Transit District')>>count(_.type_of_work)

,type_of_work,n


In [61]:
df>>filter(_.primary_agency_name=='San Joaquin Regional Transit District')>>count(_.prefix)

,prefix,n


### Bottom 50 Agencies with lowest funds

In [62]:
avg_funds_bottom = (df>>group_by(_.primary_agency_name)>>summarize(avg_funds=_.total_requested.mean())>>arrange(-_.avg_funds)).tail(50)



In [63]:
avg_funds_bottom

,primary_agency_name,avg_funds
39,Roseville,"877,868.99"
40,Sacramento,"803,820.36"
7,Capital Southeast Connector,"759,034.31"
12,Davis,"750,910.52"
37,Rancho Cordova,"734,859.60"
22,Lincoln,"723,955.43"
16,Elk Grove,"716,788.26"
6,Caltrans,"714,147.43"
50,Truckee,"671,955.93"
13,Department of Parks and Recreation,"670,868.59"


In [64]:
_dla_utils.basic_bar_chart((avg_funds_bottom.tail(40)), 'primary_agency_name','avg_funds', 'primary_agency_name', subset, 
                          'Lowest Average Total Funds by Agency')

WARN Symbol legend count exceeds limit, filtering items.


alt.Chart(...)

In [65]:
((
    _dla_utils.calculate_data_all(
        df, 'total_requested', 'primary_agency_name', aggfunc="mean"))
>>arrange(-_.total_requested)).tail(50)

,primary_agency_name,total_requested
17,Folsom,"1,239,564.15"
43,Sacramento Metropolitan Air Quality Management...,"1,187,538.65"
42,Sacramento County,"1,068,964.51"
53,West Sacramento,"1,007,069.30"
0,Auburn,"995,759.76"
49,Tahoe Transportation District,"892,835.12"
39,Roseville,"877,868.99"
40,Sacramento,"803,820.36"
7,Capital Southeast Connector,"759,034.31"
12,Davis,"750,910.52"


In [92]:
_dla_utils.basic_bar_chart((((_dla_utils.calculate_data_all(df, 'total_requested', 'primary_agency_name', aggfunc="mean"))
                          >>arrange(-_.total_requested)).tail(50)
                        ), 'primary_agency_name','total_requested', 'total_requested', subset,
                           'Average Total Requested Funds by Agency'
                       )

alt.Chart(...)

In [67]:
df.sample()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
6408,Obligated,CML,5182(063),Roseville,2016-06-14,2016-06-14,2016-06-27,2016-06-27,2016-07-05,"767,044.00",...,PLA25469,Lincoln Street To Royer Park,Extend Class I Bike Path,3,NaT,NaT,SACOG,5182,"2,016.00",Roseville


## Prefix Code Funding 

In [68]:
_dla_utils.basic_bar_chart((((_dla_utils.calculate_data_all(df, 'total_requested', 'prefix', aggfunc="mean"))
                          >>arrange(-_.total_requested)).head(30)
                        ), 'prefix','total_requested', 'prefix', subset,
                           'Average Total Requested Funds by Prefix'
                       )

alt.Chart(...)

In [69]:
_dla_utils.basic_bar_chart((((_dla_utils.calculate_data_all(df, 'fed_requested', 'prefix', aggfunc="mean"))
                          >>arrange(-_.fed_requested)).head(30)
                        ), 'prefix','fed_requested', 'prefix', subset,
                           'Average Federal Requested Funds by Prefix'
                       )

alt.Chart(...)

In [70]:
_dla_utils.basic_bar_chart((((_dla_utils.calculate_data_all(df, 'ac_requested', 'prefix', aggfunc="mean"))
                          >>arrange(-_.ac_requested)).head(30)
                        ), 'prefix','ac_requested', 'prefix', subset,
                           'Average Advanced Construction Requested Funds by Prefix'
                       )

alt.Chart(...)

## Functions to look by Prefix and by Agency: 

### By Agency:

In [71]:
_dla_utils.interactive_widget(df, 'primary_agency_name')

Dropdown(description='Primary_Agency_Name', options=('Auburn', 'Biggs', 'Butte County', 'Butte County Associat…

Output()

### By Prefix:

In [72]:
_dla_utils.interactive_widget(df, 'prefix')

Dropdown(description='Prefix', options=('ACSTER', 'ACSTP', 'ATPCML', 'ATPL', 'ATPLNI', 'ATPSB1L', 'ATPSTC', 'A…

Output()